In [1]:
%useLatestDescriptors
%use dataframe(1.0.0-Beta3), kandy

In [2]:
import org.apache.poi.ss.usermodel.Cell
import org.apache.poi.ss.usermodel.Sheet
import java.time.format.DateTimeFormatter

val formatter = DateTimeFormatter.ofPattern("yyyy-MM-dd")

fun Sheet.get(row: Int, col: Int): Cell? {
    return this.getRow(row - 1)?.getCell(col - 1)
}


In [3]:
import ds.types.raw.BoardingPass
import org.apache.poi.ss.usermodel.WorkbookFactory
import java.io.File

fun parse(file: File): Sequence<BoardingPass> {
    println("Parsing ${file.name}")
    if (file.isDirectory)
        return emptySequence()
    val records = ArrayList<BoardingPass>()
    var i = 0
    WorkbookFactory.create(file).use { wb ->
        wb.sheetIterator().forEach { sheet ->
            records.add(BoardingPass(
                name = sheet.get(3, 2)!!.stringCellValue,
                flight = sheet.get(5, 1)!!.stringCellValue,
                from = sheet.get(5, 4)!!.stringCellValue,
                to = sheet.get(5, 8)!!.stringCellValue,
                fromCode = sheet.get(7, 4)!!.stringCellValue,
                toCode = sheet.get(7, 8)!!.stringCellValue,
                date = sheet.get(9, 1)?.stringCellValue?.let { java.time.LocalDate.parse(it, formatter) },
                time = sheet.get(9, 3)?.stringCellValue?.let { java.time.LocalTime.parse(it)},
                operatedBy = sheet.get(9, 5)!!.stringCellValue,
                seat = sheet.get(11, 8)!!.stringCellValue.let {
                    if (it == "N/A")
                        return@let null
                    else
                        return@let it
                }.toString(),
                etiket = sheet.get(13, 5)!!.stringCellValue,
                pnr = sheet.get(13, 2)!!.stringCellValue,
                sequence = sheet.get(1, 8)!!.numericCellValue.toInt()
            ))
        }
    }
    return records.asSequence()
}

In [4]:
import org.apache.poi.openxml4j.util.ZipSecureFile

var res = sequenceOf<BoardingPass>()
val file = File("../data/YourBoardingPassDotAero")
ZipSecureFile.setMaxFileCount(100000)
file.walk()
    .forEach {
        res += parse(it)
    }


2025-10-28T15:48:45.308793Z Execution of code 'import org.apache.po...' ERROR Log4j API could not find a logging provider.
Parsing YourBoardingPassDotAero
Parsing YourBoardingPassDotAero-2017-01-01.xlsx
Parsing YourBoardingPassDotAero-2017-01-02.xlsx
Parsing YourBoardingPassDotAero-2017-01-03.xlsx
Parsing YourBoardingPassDotAero-2017-01-04.xlsx
Parsing YourBoardingPassDotAero-2017-01-05.xlsx
Parsing YourBoardingPassDotAero-2017-01-06.xlsx
Parsing YourBoardingPassDotAero-2017-01-07.xlsx
Parsing YourBoardingPassDotAero-2017-01-08.xlsx
Parsing YourBoardingPassDotAero-2017-01-09.xlsx
Parsing YourBoardingPassDotAero-2017-01-10.xlsx
Parsing YourBoardingPassDotAero-2017-01-11.xlsx
Parsing YourBoardingPassDotAero-2017-01-12.xlsx
Parsing YourBoardingPassDotAero-2017-01-13.xlsx
Parsing YourBoardingPassDotAero-2017-01-14.xlsx
Parsing YourBoardingPassDotAero-2017-01-15.xlsx
Parsing YourBoardingPassDotAero-2017-01-16.xlsx
Parsing YourBoardingPassDotAero-2017-01-17.xlsx
Parsing YourBoardingPassDotAe

In [5]:
import ds.utils.db.inserter

inserter<BoardingPass>().insert(res)